In [36]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as random
from scipy.signal import convolve
import scipy
from joblib import Parallel, delayed
import time
from scipy import sparse
import json
from tqdm import tqdm, tqdm_notebook
import os

%run ../Scripts/coverage.py
%run ../Scripts/fitness.py
%run ../Scripts/immunity.py
%run ../Scripts/initMethods.py
%run ../Scripts/supMethods.py
%run ../Scripts/mutation.py

In [37]:
params = { #parameters relevant for the equations
    "Nh":         1000000,
    "N0":          100000, #This Will be updated by self-consitent solution
    "R0":              20, 
    "M":                1, #Also L, total number of spacers
    "mu":            0.01, #mutation rate
    "gamma_shape":     20, 
    "Np":               0, #Number of Cas Protein
    "dc":               3, #Required number of complexes to activate defence
    "h":                4, #coordination coeff
    "r":             2000, #cross-reactivity kernel
}
sim_params = { #parameters relevant for the simulation (including Inital Valuess)
    "xdomain":                   1000,
    "dx":                           1,
    "tf":                        1000,
    "dt":                           1,
    "initial_mean_n":           [0,0],
    "initial_mean_nh":          [0,0],
    "conv_size":                 4000,
    "num_threads":                 32,
    "tail_axis":               [1, 1],
}

In [38]:
params, sim_params = init_cond(params, sim_params)
foldername = "test1/"
try:
    write2json(foldername, params, sim_params)
except FileNotFoundError:
    os.mkdir(foldername)
    write2json(foldername, params, sim_params)


Phage Population: 100000.0000| Uc: 7.7871| sigma: 2.4223
Phage Population: 529.5384| Uc: 3.0017| sigma: 1.9086
Phage Population: 328.7707| Uc: 2.3910| sigma: 1.8031
Phage Population: 293.4287| Uc: 2.2338| sigma: 1.7727
Phage Population: 283.6152| Uc: 2.1857| sigma: 1.7631
Phage Population: 280.5440| Uc: 2.1701| sigma: 1.7600
Phage Population: 279.5467| Uc: 2.1651| sigma: 1.7589
Phage Population: 279.2189| Uc: 2.1623| sigma: 1.7584


In [39]:
n = init_guassian(params["N"], sim_params, "n")
nh = init_exptail(params["Nh"], params, sim_params, "nh")


In [ ]:
plt.figure()
plt.contour(n.todense(), cmap = "Reds")
plt.contour(nh.todense(), cmap = "Blues")
plt.margins(-0.4, -0.4)
plt.show()

In [14]:
n = init_guassian(params["N0"], sim_params, "n")
nh = init_exptail(params["Nh"], params, sim_params, "nh")
kernel_quarter = init_quarter_kernel(params, sim_params)

for i in range(sim_params["tf"]):
    sparse.save_npz(foldername+f"sp_frame_n{i}",n.tocoo())
    sparse.save_npz(foldername+f"sp_frame_nh{i}",nh.tocoo())

    p = elementwise_coverage(nh, n, kernel, params, sim_params)
    f = fitness_spacers(n, nh, p, params, sim_params) #f is now a masked array (where mask is where eff_R0 = 0)
    f = norm_fitness(f, n, params, sim_params) #renormalize f
    n = virus_growth(n, f, params, sim_params) #update

    n = mutation(n, params, sim_params)
    nh = immunity_update(nh, n, params, sim_params) #update nh

/Users/zhizhang144515/Documents/CRISPR_Sim/Scripts/initMethods.py:117: RuntimeWarning: overflow encountered in exp
  p_marg_y = np.exp(-y_linspace/(v0*tau))*np.heaviside(axis[1]*(y_linspace-initial_position[1]), 0)
/Users/zhizhang144515/Documents/CRISPR_Sim/Scripts/initMethods.py:117: RuntimeWarning: invalid value encountered in multiply
  p_marg_y = np.exp(-y_linspace/(v0*tau))*np.heaviside(axis[1]*(y_linspace-initial_position[1]), 0)


ValueError: probabilities contain NaN

In [ ]:
R0_sweep = [20]
kernel = init_quarter_kernel(params, sim_params)
foldername = "test0/"
write2json(foldername, params, sim_params)

for R0 in R0_sweep:
    params["R0"] = R0
    n = init_guassian(params["N"], sim_params, "n")
    nh = init_guassian(params["Nh"], sim_params, "nh")

    for i in range(10000):

        if (i%100 == 0 or i < 20):
            sparse.save_npz(foldername+f"sp_frame_n{i}",n.tocoo())
            sparse.save_npz(foldername+f"sp_frame_nh{i}",nh.tocoo())

        p = elementwise_coverage(nh, n, kernel, params, sim_params)
        f = fitness_spacers(n, nh, p, params, sim_params) #f is now a masked array (where mask is where eff_R0 = 0)
        # f = norm_fitness(f, n, params, sim_params)
        n = virus_growth(n, f, params, sim_params) #update

        n = mutation(n, params, sim_params)
        nh = immunity_update(nh, n, params, sim_params) #update nh

        if np.sum(n) >= 10*params["Nh"]:
            break